In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import h5py
import keras.backend as K
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Activation
from keras import optimizers, layers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


## Productivity improvement (CB1,CB2,CB4,CB5,CB7)

In [2]:
x_1 = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_train_scale.csv', 
                    delimiter = ',', usecols = (2,3,4,5,6), skip_header=2)
y_1o = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_train_scale.csv', 
                   delimiter =',', usecols = (29), skip_header=2).reshape (54,1)
x_2 = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_cv_scale.csv', 
                    delimiter = ',', usecols = (2,3,4,5,6), skip_header=1)
y_2o = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_cv_scale.csv',
                      delimiter =',', usecols = (29),skip_header=1).reshape (14,1)

In [3]:
enc = OneHotEncoder ()
enc.fit (y_1o)
y_1 = enc.transform(y_1o).toarray()
y_2 = enc.transform(y_2o).toarray()

In [4]:
y_whole = np.vstack ((y_1, y_2)).astype(np.float32)
x_whole = np.vstack ((x_1, x_2)).astype(np.float32)
x_traind = x_1
x_test = x_2
y_traind = y_1
y_test = y_2
x_train, x_cv, y_train, y_cv = train_test_split (x_1,y_1,test_size=0.13)

In [5]:
num_classes = 4;

model = Sequential ()
model.add (Dense (units = 11, input_dim = 5))
model.add (layers.Activation ('tanh'))
model.add (Dense (units = 11))
#model.add (layers.Activation ('tanh'))
#model.add (Dense (units = 11))
#model.add (layers.Activation ('tanh'))
#model.add (Dense (units = 16))
model.add (Dense(num_classes, activation = 'softmax'))

#sgd = optimizer.SGD (lr=0.01, momentum = 0, decay =0, nestrov = False)
model.compile (loss = 'binary_crossentropy', optimizer = 'sgd', 
               metrics=['binary_accuracy'])

History = model.fit(x_train, y_train,
                    batch_size=15,
                    epochs=40,
                    verbose=1, validation_data=(x_cv, y_cv))

print(model.evaluate (x_test, y_test, batch_size = 15, verbose =1))

#help (model.evaluate)
#this is the output units

y_pred_train = model.predict (x_train)
y_pred_cv = model.predict (x_cv)
y_pred_test = model.predict (x_test)
y_pred_whole = np.vstack ((y_pred_train, y_pred_cv, y_pred_test)).astype(np.float32)

Train on 46 samples, validate on 8 samples
Epoch 1/40
46/46 [==============================] - ETA: 0s - loss: 0.6688 - binary_accuracy: 0.6576 - val_loss: 0.5029 - val_binary_accuracy: 0.7500
Epoch 2/40
46/46 [==============================] - ETA: 0s - loss: 0.6618 - binary_accuracy: 0.6576 - val_loss: 0.5012 - val_binary_accuracy: 0.7500
Epoch 3/40
46/46 [==============================] - ETA: 0s - loss: 0.6595 - binary_accuracy: 0.6576 - val_loss: 0.4984 - val_binary_accuracy: 0.7500
Epoch 4/40
46/46 [==============================] - ETA: 0s - loss: 0.6554 - binary_accuracy: 0.6576 - val_loss: 0.4960 - val_binary_accuracy: 0.7500
Epoch 5/40
46/46 [==============================] - ETA: 0s - loss: 0.6529 - binary_accuracy: 0.6576 - val_loss: 0.4935 - val_binary_accuracy: 0.7500
Epoch 6/40
46/46 [==============================] - ETA: 0s - loss: 0.6493 - binary_accuracy: 0.6630 - val_loss: 0.4905 - val_binary_accuracy: 0.7500
Epoch 7/40
46/46 [==============================] - ETA: 

### t(x) for labelling

In [6]:
#save model to json, weights to HDF5
model_json = model.to_json ()
with open ('model_lod_productiviity_improve_a.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model_lod_productiviity_improve_a.h5')
print ('Saved model to disk')

Saved model to disk


In [5]:
#load json and create model
json_file = open('model_lod_productiviity_improve_a.json', 'r')
loaded_model_json = json_file.read ()
json_file.close ()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('model_lod_productiviity_improve_a.h5')

In [6]:
loaded_model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', 
               metrics=['binary_accuracy'])
print(loaded_model.evaluate (x_test, y_test, batch_size = 20, verbose =1))
y_pred_test = loaded_model.predict (x_test)

                     
i = 0;
j = 0;
m = np.zeros((14,4));
for i in range (14):
    max = 0.0000001;
    for j in range (4):
        if y_pred_test.item ((i,j))>=max:
            max = y_pred_test.item ((i,j))
    for j in range (4):
        if y_pred_test.item ((i,j)) ==max:
            np.put(m, [4*i+j], [1])

i = 0;
j = 0;
selected_LOD = np.zeros((14,1));
for i in range (14):
    for j in range (4):
        if m.item((i,j))>0:
            selected_LOD_1 = m.item((i,j))*y_pred_test.item((i,j));
            np.put(selected_LOD, [i], [selected_LOD_1])

i = 0;
j = 0;
cf = np.zeros((14,4))
for i in range (14):
    for j in range (4):
        cf_1 = y_pred_test.item((i,j))-selected_LOD.item((i,0));
        cf_2 = 10*cf_1;
        np.put(cf, [4*i+j], [cf_2])
print (cf)        
np.savetxt('y_cost_LODproduct_improve.csv', cf, delimiter=',')

14/14 [==============================] - ETA: 0s
[0.56513655185699463, 0.75]
[[-5.42719297  0.         -4.82325315 -4.47360173]
 [-4.20982778 -4.951462    0.         -3.21113944]
 [-0.88172376 -1.52010173 -2.79351346  0.        ]
 [ 0.         -2.21382469 -1.83994904 -0.3964141 ]
 [-2.96225809 -2.29478329  0.         -0.38900256]
 [-0.65744609  0.         -0.98634332 -0.08596033]
 [-0.67225248 -1.80695876 -2.50010826  0.        ]
 [-1.19168922 -0.36519259  0.         -0.19582748]
 [-0.78011364 -0.02923638  0.         -0.12578428]
 [-0.37026063 -0.46207145 -0.38329333  0.        ]
 [-2.96496138 -2.7046594  -3.24551404  0.        ]
 [-3.88583012 -3.42166677  0.         -2.16614574]
 [-1.24191999 -2.35340625 -0.90603322  0.        ]
 [-3.31119619 -2.03805998 -0.72856218  0.        ]]
